In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import time
import numpy as np
from six import iteritems
import shelve
from scipy.ndimage.measurements import label
import h5py

import PIL
import os
from IPython.display import Image, display
from IPython.html.widgets import interact, fixed
from IPython.html import widgets
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

import h5py
from skimage import feature
import os
from os.path import expanduser
import time

import torch

from rollback_pyramid import RollbackPyramid
from alignermodule import Aligner
from stack_dataset import compile_dataset

def visualize_residuals(res, figsize=(10,10), x_coords=None, y_coords=None, vec_grid=50):
    assert res.shape[0] == res.shape[1]
    plt.figure(figsize=figsize)
    n = res.shape[0]
    y, x = np.mgrid[0:n, 0:n]
    
    if x_coords is None:
        x_coords = [0, res.shape[0]]
    if y_coords is None:
        y_coords = [0, res.shape[1]]
    
    ex = (-1) * res[:, :, 0]
    ey = res[:, :, 1]
    r = np.arctan2(ex, ey)
    
    interval = (x_coords[1] - x_coords[0]) // vec_grid
    
    plt.quiver(  x[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval],  
                 y[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval],
                ex[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval], 
                ey[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval], 
                 r[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval], alpha=0.6)
    plt.quiver(x[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval],  
                 y[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval],
                ex[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval], 
                ey[x_coords[0]:x_coords[1]:interval, y_coords[0]:y_coords[1]:interval], edgecolor='k', facecolor='None', linewidth=.5)
    plt.gca().invert_yaxis()

def display_image(img, x_coords=None, y_coords=None, normalize=False, figsize=(10, 10), mask=False):
    if normalize and mask:
        raise Exception("Masks can't be normalized")
    
    plt.figure(figsize=figsize)
    
    if x_coords is None:
        x_coords = [0, img.shape[0]]
    if y_coords is None:
        y_coords = [0, img.shape[1]]
    
    if mask:
        plt.imshow(img[x_coords[0]:x_coords[1], y_coords[0]:y_coords[1]], cmap='gray', vmin=0, vmax=1.0)
    elif not normalize:
        plt.imshow(img[x_coords[0]:x_coords[1], y_coords[0]:y_coords[1]], cmap='gray', vmin=-0.5, vmax=0.5)
    else:
        plt.imshow(img[x_coords[0]:x_coords[1], y_coords[0]:y_coords[1]], cmap='gray')

class simple_visualizer():
    def __init__(self):
        pass
    
    def display_multiimg(self, choice):
        i = self.names.index(choice)
        if isinstance(self.images[i], torch.Tensor):
            self.images[i] = self.images[i].cpu().detach().numpy()
        self.images[i] = self.images[i].squeeze()
        
        if self.crop == 0:
            display_image(self.images[i], normalize=self.normalize, figsize=(12, 12))
        else:
            display_image(self.images[i][self.crop:-self.crop, self.crop:-self.crop], 
                          normalize=self.normalize, figsize=(12, 12))
        
    def visualize(self, images, names=None, crop=0, normalize=False):
        if names is None:
            names = range(len(images))
        self.names = names
        self.normalize = normalize
        self.images = images
        self.crop = crop
        
        button_choice = widgets.ToggleButtons(
                options=names,
                description='Image:',
                disabled=False,
                button_style='',
            )

        interact(self.display_multiimg, choice=button_choice)

/usr/people/popovych/.local/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


#  Dataset

In [3]:
dataset = compile_dataset([{"data": {"h5": "./datasets/minnie_full_mip8_1300px_train.h5", "mip": 8}, 
                                            "masks": {
                                                "plastic": {"h5": "./datasets/plastic_mask_minnie_full_mip8_1300px_train.h5", "mip": 8}
                                            }}], supervised=False, max_size=500)

plastic mask detected!


### Compose Pyramid

In [4]:
aligners = {}
pyramid = RollbackPyramid()

for m in [8, 9, 10]:
    aligners[m] = Aligner(fms=[2, 16, 16, 16, 16, 2], k=7).cuda()
    aligners[m].load_state_dict(torch.load('./checkpoints/barak_aligner_mip{}.pth.tar'.format(m)))
    pyramid.set_mip_processor(aligners[m], m)
    

In [ ]:
#torch.save(model.mip_processors[8].state_dict(), "./barak_aligner_mip8.pth.tar")

In [7]:
i = 0
sample = torch.FloatTensor(dataset[i].unsqueeze(0) * 1)
src_tgt = sample[:, 0:2]
plastic_mask = sample[:, 8:10]
src_var, tgt_var, pred_res, pred_tgt = pyramid.run_pair(sample, plastic_mask, 8)



AttributeError: 'numpy.ndarray' object has no attribute 'unsqueeze'

In [ ]:
print (pred_tgt.shape)
viz = [src_var[0], tgt_var[0], pred_tgt]
simple_visualizer().visualize(viz, normalize=True)